# Tuto Neo4j

Le but du tutoriel est de vous faire découvrir Neo4j, une base de données orientée graphe. Nous allons voir comment installer Neo4j, créer une base de données, insérer des données et effectuer des requêtes simples.

On utilisera pour ce tutoriel un container neo4j, ainsi que le dataset suivant : [Social circles: Facebook](https://snap.stanford.edu/data/ego-Facebook.html).

On utilisera les librairies python `py2neo` pour interagir avec Neo4j. Cependant, si l'envie vous vient de faire un projet en Neo4j, utilisez neomodel qui est l'ORM de Neo4j.

### Qu'est ce que Neo4j ?

Neo4j est une base de données orientée graphe qui permet de stocker et de manipuler des données sous forme de graphes. Un graphe est composé de nœuds (ou sommets) et de relations (ou arêtes) entre ces nœuds. Neo4j utilise le langage de requête Cypher pour interagir avec les données.

Une fois que Neo4j est installé, vous pouvez accéder à l'interface web de Neo4j en ouvrant votre navigateur et en allant à l'adresse `http://localhost:7474`. Utilisez les identifiants suivant pour vous connecter : neo4j / strongpassword .

### Création de la base de données

Une fois connecté à l'interface web de Neo4j, vous vous retrouverez sur cette page d'accueil :
![Page d'accueil](./images/screen_1.png "Accueil")

On voit un prompt dans lequel on peut écrire des requêtes Cypher. Commençons par lister les bases de données existantes avec la requête suivante :
```cypher
SHOW DATABASES;
```
Combien de bases de données voyez-vous ? Par défaut, Neo4j crée une base de données nommée `neo4j`. Nous allons créer une nouvelle base de données pour notre tutoriel. Utilisez la requête suivante pour créer une base de données nommée `social` :
```cypher
CREATE DATABASE social;
```

### Création base de données social pour le tp

In [1]:
from py2neo import Graph

sys = Graph("bolt://neo4j:7687", auth=("neo4j", "strongpassword"), name="system")
sys.run("CREATE DATABASE social IF NOT EXISTS WAIT;")

address,state,message,success
,CaughtUp,No operation needed,true


### Insertion des données

On va télécharger les données via un curl, et les charger dans Neo4j avec python.

In [2]:
!wget https://snap.stanford.edu/data/facebook.tar.gz && tar -xvzf facebook.tar.gz && rm facebook.tar.gz

--2025-11-21 12:32:51--  https://snap.stanford.edu/data/facebook.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 732104 (715K) [application/x-gzip]
Saving to: ‘facebook.tar.gz’

facebook.tar.gz     100%[===================>] 714.95K   336KB/s    in 2.1s    

2025-11-21 12:32:54 (336 KB/s) - ‘facebook.tar.gz’ saved [732104/732104]

facebook/
facebook/3980.egofeat
facebook/0.featnames
facebook/698.egofeat
facebook/3437.feat
facebook/3980.featnames
facebook/0.edges
facebook/3437.circles
facebook/686.circles
facebook/348.egofeat
facebook/107.feat
facebook/348.feat
facebook/1912.circles
facebook/3437.egofeat
facebook/698.feat
facebook/348.edges
facebook/1912.feat
facebook/414.circles
facebook/1684.egofeat
facebook/1684.featnames
facebook/1684.feat
facebook/107.egofeat
facebook/0.circles
facebook/414.edges
facebook/698.featnames
fac

On voit que dans le dossier facebook, on a plusieurs fichier avec des extensions différentes :
- `.egofeat` : Caractéristiques des utilisateurs (nœuds)
- `.edges` : Relations entre les utilisateurs (arêtes)
- `.feat` : Caractéristiques des relations (arêtes)
- `.featnames` : Noms des caractéristiques des relations
- `.circles` : Cercles d'amis (groupes d'utilisateurs)


In [10]:
from py2neo import Graph

graph = Graph("bolt://neo4j:7687", auth=("neo4j", "strongpassword"), name="social")
graph.run("RETURN 'Connected to Neo4j!' AS message").to_data_frame()

,message
0,Connected to Neo4j!


### Import de tous les egos

Maintenant, nous allons charger les données de tous les egos disponibles dans le dataset. Chaque ego représente un utilisateur central avec son réseau d'amis. Pour différencier les egos, nous allons créer un nœud `Ego` pour chaque utilisateur central et lier les personnes à leur ego respectif.

In [4]:
import glob
import os
from tqdm import tqdm
from py2neo import Node
import pandas as pd

# Récupérer tous les IDs d'egos
ego_files = glob.glob('facebook/*.edges')
ego_ids = sorted([os.path.basename(f).replace('.edges', '') for f in ego_files])

print(f"Nombre d'egos trouvés : {len(ego_ids)}")
print(f"IDs des egos : {ego_ids}")

# Fonction pour charger un ego complet
def load_ego(ego_id, graph):
    print(f"\n{'='*60}")
    print(f"Chargement de l'ego {ego_id}")
    print(f"{'='*60}")
    
    # Charger les noms de caractéristiques
    feature_names = []
    featnames_file = f'facebook/{ego_id}.featnames'
    if os.path.exists(featnames_file):
        with open(featnames_file, 'r') as f:
            for line in f:
                parts = line.strip().split(' ', 1)
                if len(parts) == 2:
                    _, name = parts
                    feature_names.append(name.strip())
    
    # Créer le nœud Ego
    ego_node = Node("Ego", id=int(ego_id), name=f"Ego-{ego_id}")
    graph.merge(ego_node, "Ego", "id")
    
    # Charger les caractéristiques de l'ego (egofeat)
    egofeat_file = f'facebook/{ego_id}.egofeat'
    if os.path.exists(egofeat_file):
        with open(egofeat_file, "r") as f:
            line = f.readline()
            if line.strip():
                parts = line.strip().split()
                features_dict = {}
                for j in range(len(parts)):
                    key = feature_names[j] if j < len(feature_names) else f'f{j}'
                    features_dict[key] = int(parts[j])
                query = """
                MATCH (e:Ego {id:$id})
                SET e += $features
                """
                graph.run(query, id=int(ego_id), features=features_dict)
    
    # Charger les features (nœuds du réseau de l'ego)
    feat_file = f'facebook/{ego_id}.feat'
    if os.path.exists(feat_file):
        features_df = pd.read_csv(feat_file, sep=' ', header=None)
        print(f"{len(features_df)} personnes dans le réseau")
        
        for i, row in tqdm(features_df.iterrows(), total=features_df.shape[0], desc=f"Nœuds"):
            node_id = int(row[0])
            node_features = {}
            for j in range(1, len(row)):
                key = feature_names[j-1] if j-1 < len(feature_names) else f'f{j-1}'
                node_features[key] = int(row[j])
            
            person = Node("Person", id=node_id, name=str(node_id), ego_id=int(ego_id), **node_features)
            graph.merge(person, "Person", "id")
            
            # Lier la personne à son ego
            query_ego_link = """
            MATCH (p:Person {id: $person_id}), (e:Ego {id: $ego_id})
            MERGE (p)-[:BELONGS_TO_EGO]->(e)
            """
            graph.run(query_ego_link, person_id=node_id, ego_id=int(ego_id))
    
    # Charger les edges (relations entre personnes)
    edges_file = f'facebook/{ego_id}.edges'
    if os.path.exists(edges_file):
        edges_df = pd.read_csv(edges_file, sep=' ', header=None, names=['source', 'target'])
        print(f"{len(edges_df)} relations d'amitié")
        
        for i, row in tqdm(edges_df.iterrows(), total=edges_df.shape[0], desc=f"Relations"):
            src, dst = int(row["source"]), int(row["target"])
            query = """
            MATCH (a:Person {id: $src}), (b:Person {id: $dst})
            MERGE (a)-[:FRIEND_WITH]->(b)
            """
            graph.run(query, src=src, dst=dst)
    
    # Charger les cercles
    circles_file = f'facebook/{ego_id}.circles'
    if os.path.exists(circles_file):
        circles = []
        with open(circles_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) > 1:
                    circle_name = parts[0]
                    members = [int(x) for x in parts[1:]]
                    circles.append({"circle": circle_name, "members": members})
        
        if circles:
            print(f"{len(circles)} cercles")
            for c in tqdm(circles, desc=f"Cercles"):
                circle_name = c["circle"]
                for member_id in c["members"]:
                    graph.run("""
                    MATCH (p:Person {id:$id})
                    MERGE (c:Circle {name:$circle, ego_id:$ego_id})
                    MERGE (p)-[:IN_CIRCLE]->(c)
                    """, id=member_id, circle=circle_name, ego_id=int(ego_id))
    
    print(f"Ego {ego_id} chargé avec succès")
    return True

Nombre d'egos trouvés : 10
IDs des egos : ['0', '107', '1684', '1912', '3437', '348', '3980', '414', '686', '698']


Chargement de tous les egos

In [5]:
# Charger tous les egos dans Neo4j
print(f"Début du chargement de {len(ego_ids)} egos...\n")

for ego_id in ego_ids:
    try:
        load_ego(ego_id, graph)
    except Exception as e:
        print(f"Erreur lors du chargement de l'ego {ego_id}: {e}")
        continue

print(f"\n{'='*60}")
print(f"Tous les egos ont été chargés dans Neo4j!")
print(f"{'='*60}")

Début du chargement de 10 egos...


Chargement de l'ego 0
347 personnes dans le réseau


Nœuds: 100%|██████████| 347/347 [00:07<00:00, 44.53it/s] 


5038 relations d'amitié


Relations: 100%|██████████| 5038/5038 [00:30<00:00, 167.03it/s]


24 cercles


...

540 relations d'amitié


Relations: 100%|██████████| 540/540 [00:01<00:00, 419.37it/s]


13 cercles


Cercles: 100%|██████████| 13/13 [00:00<00:00, 79.06it/s]


Ego 698 chargé avec succès

Tous les egos ont été chargés dans Neo4j!


### Vérification des données importées

Vérifions maintenant combien d'egos, de personnes et de relations ont été importés dans Neo4j :

In [ ]:
ego_ids

['0', '107', '1684', '1912', '3437', '348', '3980', '414', '686', '698']

In [ ]:
# Statistiques globales
stats = graph.run("""
MATCH (e:Ego)
WITH count(e) as num_egos
MATCH (p:Person)
WITH num_egos, count(p) as num_persons
MATCH ()-[r:FRIEND_WITH]->()
WITH num_egos, num_persons, count(r) as num_friendships
MATCH (c:Circle)
RETURN num_egos as Egos, num_persons as Personnes, num_friendships as Amitiés, count(c) as Cercles
""").to_data_frame()

print("Statistiques globales de la base de données:")
display(stats)

Statistiques globales de la base de données:


,Egos,Personnes,Amitiés,Cercles
0,10,4035,168486,193


In [ ]:
# Détail par ego
ego_details = graph.run("""
MATCH (e:Ego)<-[:BELONGS_TO_EGO]-(p:Person)
WITH e, count(p) as num_persons
OPTIONAL MATCH (p2:Person {ego_id: e.id})-[:FRIEND_WITH]->()
WITH e, num_persons, count(p2) as num_friendships
OPTIONAL MATCH (c:Circle {ego_id: e.id})
WITH e, num_persons, num_friendships, count(c) as num_circles
RETURN e.id as EgoID, num_persons as Personnes, num_friendships as Amitiés, num_circles as Cercles
ORDER BY e.id
""").to_data_frame()

print("Détail par ego:")
display(ego_details)

Détail par ego:


,EgoID,Personnes,Amitiés,Cercles
0,0,347,4998,24
1,107,1045,51963,9
2,348,227,4330,14
3,414,159,5332,7
4,686,170,2438,14
5,698,66,1155,13
6,1684,792,28218,17
7,1912,755,60141,46
8,3437,547,9622,32
9,3980,59,289,17


### Exemples de requêtes sur tous les egos

Maintenant que tous les egos sont chargés, explorons les données avec quelques requêtes :

In [ ]:
# Trouver les personnes les plus connectées tous egos confondus
top_connected = graph.run("""
MATCH (p:Person)-[:FRIEND_WITH]->()
WITH p, count(*) as num_friends
RETURN p.id as PersonID, p.ego_id as EgoID, num_friends as NombreAmis
ORDER BY num_friends DESC
LIMIT 10
""").to_data_frame()

print("Top 10 des personnes les plus connectées:")
display(top_connected)

Top 10 des personnes les plus connectées:


,PersonID,EgoID,NombreAmis
0,2543,1912,293
1,2347,1912,290
2,1888,107,253
3,1800,107,244
4,1663,107,234
5,1352,107,233
6,2266,1912,233
7,483,414,231
8,1730,107,225
9,1985,1912,223


In [ ]:
# Comparer les réseaux des différents egos
ego_comparison = graph.run("""
MATCH (e:Ego)
OPTIONAL MATCH (e)<-[:BELONGS_TO_EGO]-(p:Person)
WITH e, count(p) as network_size
OPTIONAL MATCH (p2:Person {ego_id: e.id})-[r:FRIEND_WITH]->()
WITH e, network_size, count(r) as total_connections
RETURN e.id as EgoID, 
       network_size as TailleRéseau, 
       total_connections as Connexions,
       CASE WHEN network_size > 0 THEN round(toFloat(total_connections) / network_size, 2) ELSE 0 END as Moyenne_connexions
ORDER BY network_size DESC
""").to_data_frame()

print("Comparaison des réseaux d'egos:")
display(ego_comparison)

Comparaison des réseaux d'egos:


,EgoID,TailleRéseau,Connexions,Moyenne_connexions
0,107,1045,51963,49.73
1,1684,792,28218,35.63
2,1912,755,60141,79.66
3,3437,547,9622,17.59
4,0,347,4998,14.40
5,348,227,4330,19.07
6,686,170,2438,14.34
7,414,159,5332,33.53
8,698,66,1155,17.50
9,3980,59,289,4.90


### Visualisation des données dans Neo4j

L'interêt de Neo4j est de pouvoir visualiser les données sous forme de graphe. Pour cela, on peut utiliser l'interface web de Neo4j. Voici comment visualiser les amis de la personne avec l'ID 0 :
```cypher
MATCH (p:Person {id:0})-[:FRIEND_WITH]->(friend)
RETURN p, friend
```
Et voilà le résultat :
![Visualisation des amis](./images/screen_2.png "Amis de la personne 0")

En double cliquant sur le noeud 0, on peut voir les rélations du noeud et ses caractéristiques :
![Détails du noeud](./images/screen_3.png "Détails du noeud 0")

In [ ]:
# Trouver le chemin le plus court entre deux personnes (Cypher)
result = graph.run("""
MATCH path = shortestPath((p1:Person {id: 10})-[*]-(p2:Person {id: 100}))
RETURN length(path) as distance, 
       [node in nodes(path) | node.id] as chemin
""").to_data_frame()

print("Chemin le plus court entre la personne 10 et la personne 100:")
display(result)


Chemin le plus court entre la personne 10 et la personne 100:


,distance,chemin
0,2,"[10, 0, 100]"


In [ ]:
# Explorer les caractéristiques d'une personne et ses amis directs
person_info = graph.run("""
MATCH (p:Person {id: 0})
OPTIONAL MATCH (p)-[:FRIEND_WITH]->(friend)
RETURN p.id as PersonneID, 
       p.ego_id as EgoID,
       count(friend) as NombreAmis,
       collect(friend.id)[0..5] as PremiersAmis
""").to_data_frame()

print("Informations sur la personne 0:")
display(person_info)


Informations sur la personne 0:


,PersonneID,EgoID,NombreAmis,PremiersAmis
0,0,107,2,"[58, 171]"


### Fonctions avancées de Cypher

Nous allons utiliser des fonctions plus complèxes de Cypher afin de voir ce que Neo4j apporte de plus par rapport à une base de données relationnelle classique. Pour cela on va utiliser GDS (Graph Data Science) qui est un plugin de Neo4j permettant d'effectuer des analyses de graphes avancées.

Questions :
- Quel est le degré moyen des nœuds dans le graphe ?
- Quelle est la distance moyenne entre deux nœuds dans le graphe ?
- Quels sont les noeuds les plus influents dans le graphe (PageRank) ?
- Quels sont les clusters dans le graphe (algorithme de Louvain) ?

In [16]:
# Supprimer le graphe s'il existe déjà et le recréer
try:
    graph.run("CALL gds.graph.drop('facebookGraph', false)")
    print("Ancien graphe supprimé")
except:
    print("Aucun graphe existant à supprimer")

# Créer la projection du graphe
result = graph.run("""
CALL gds.graph.project('facebookGraph', 'Person', 'FRIEND_WITH')
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
""").to_data_frame()

print("Graphe créé:")
display(result)

Ancien graphe supprimé
Graphe créé:


,graphName,nodeCount,relationshipCount
0,facebookGraph,4035,168486


In [17]:
# Degré moyen des nœuds
result = graph.run("""
CALL gds.degree.stream('facebookGraph')
YIELD nodeId, score
RETURN avg(score) AS average_degree
""").to_data_frame()

print("Degré moyen des nœuds:")
display(result)

Degré moyen des nœuds:


,average_degree
0,41.756134


In [18]:
# Distance moyenne entre deux nœuds (utilisation que sur un échantillon)
result = graph.run("""
MATCH (source:Person)
WITH source, id(source) AS sourceId
LIMIT 100
MATCH (target:Person)
WHERE id(target) > sourceId
WITH source, target
LIMIT 500
MATCH path = shortestPath((source)-[:FRIEND_WITH*]-(target))
RETURN avg(length(path)) AS average_distance
""").to_data_frame()

print("Distance moyenne entre les nœuds (échantillon):")
display(result)

Distance moyenne entre les nœuds (échantillon):


,average_distance
0,2.785425


In [19]:
# Quels sont les nœuds les plus influents dans le graphe (PageRank)?
result = graph.run("""
CALL gds.pageRank.stream('facebookGraph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS person_id, score
ORDER BY score DESC
LIMIT 10
""").to_data_frame()

print("Top 10 des personnes les plus influentes (PageRank):")
display(result)

Top 10 des personnes les plus influentes (PageRank):


,person_id,score
0,483,5.142148
1,3830,5.105381
2,2313,3.599598
3,376,3.560632
4,2047,3.447173
5,25,3.156732
6,828,3.124927
7,428,3.115072
8,475,3.072827
9,56,3.042849


In [20]:
# Quels sont les clusters dans le graphe (algorithme de Louvain)?
result = graph.run("""
CALL gds.louvain.stream('facebookGraph')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).id AS person_id, communityId
ORDER BY communityId
LIMIT 20
""").to_data_frame()

print("Communautés détectées par l'algorithme de Louvain:")
display(result)

Communautés détectées par l'algorithme de Louvain:


,person_id,communityId
0,911,15
1,918,22
2,953,155
3,934,155
4,921,155
5,952,155
6,926,155
7,897,155
8,932,155
9,906,155


In [21]:
# Statistiques sur les communautés
community_stats = graph.run("""
CALL gds.louvain.stream('facebookGraph')
YIELD nodeId, communityId
RETURN communityId, count(*) as members
ORDER BY members DESC
LIMIT 10
""").to_data_frame()

print("Top 10 des communautés par taille:")
display(community_stats)

Top 10 des communautés par taille:


,communityId,members
0,1905,496
1,1038,417
2,3664,416
3,312,326
4,2559,277
5,3336,264
6,1166,241
7,2390,237
8,3107,228
9,1647,226
